In [ ]:
RESULTS_FOLDER = "/path/to/results/folder"

In [ ]:
import os

import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import scipy.stats
matplotlib.rc('font', family='serif', size=24)
matplotlib.rc('errorbar', capsize=3)

In [ ]:
exp = "adult"
client_extension = os.path.join("inv_lambda-1.0_epsilon-0.01_apply_dp-True", "data_dp_round_1.npz")
epsilon_prefix = "split_sym_eq_50_validation_fraction_0.2"
def _load_client_data(exp_name):
    return {
        1: np.load(os.path.join(RESULTS_FOLDER, exp_name, "split_sym_eq_1_validation_fraction_0.2", client_extension)),
        25: np.load(os.path.join(RESULTS_FOLDER, exp_name, "split_sym_eq_25_validation_fraction_0.2", client_extension)),
        50: np.load(os.path.join(RESULTS_FOLDER, exp_name, "split_sym_eq_50_validation_fraction_0.2", client_extension)),
        100: np.load(os.path.join(RESULTS_FOLDER, exp_name, "split_sym_eq_100_validation_fraction_0.2", client_extension)),
    }
def _load_epsilon_data(exp_name):
    return {
        0.01: np.load(os.path.join(RESULTS_FOLDER, exp_name, epsilon_prefix, "inv_lambda-1.0_epsilon-0.01_apply_dp-True", "data_dp_round_1.npz")),
        0.1: np.load(os.path.join(RESULTS_FOLDER, exp_name,epsilon_prefix, "inv_lambda-1.0_epsilon-0.1_apply_dp-True", "data_dp_round_1.npz")),
        1.0: np.load(os.path.join(RESULTS_FOLDER, exp_name,epsilon_prefix, "inv_lambda-1.0_epsilon-1.0_apply_dp-True", "data_dp_round_1.npz")),
        10.0: np.load(os.path.join(RESULTS_FOLDER, exp_name,epsilon_prefix, "inv_lambda-1.0_epsilon-10.0_apply_dp-True", "data_dp_round_1.npz")),
    }
adult_client_data = _load_client_data("adult")
adult_epsilon_data = _load_epsilon_data("adult")
kdd_client_data = _load_client_data("kdd99")
kdd_epsilon_data = _load_epsilon_data("kdd99")

In [ ]:
def plot(ax, title_to_client_data, x_label, logscale = False, legend = False, ylabel=False):
    ax.grid(which='both', axis='both')
    if ylabel:
        ax.set_ylabel('Model F1\n Score ($m_k$)')
    ax.set_xlabel(x_label)
    ax.set_ybound(0.0, 1.25)
    ax.set_autoscalex_on(False)
    ax.minorticks_on()
    if logscale:
        ax.set_xscale('log')
        ax.set_xbound(10**(-2.5), 10**(1.5))
        ax.set_xticks([1e-2, 1e-1, 1, 10], minor=False)
        ax.set_xticks([], minor=True)
        ax.set_xticklabels([], minor=True)
    else:
        ax.set_xbound(-5, 105)
        ax.set_xticks([0, 50, 100], minor=False)
        ax.set_xticks([25, 75], minor=True)
    ax.set_yticks([0, 0.5, 1.0], minor=False)
    ax.set_yticks([0.25, 0.75], minor=True)
    ax.set_autoscaley_on(False)
    
    fmts = [['ok', 'yD']]  # , ['ro', 'cp']

    for fmt, (title, client_data) in zip(fmts, title_to_client_data.items()):
        ax.set_title(title)
        xs = list(client_data.keys())
        gt_ys = []
        gt_errors = []
        median_ys = []
        median_errors = []
        for data in client_data.values():
            ground_truth_scores = data['ground_truth_scores']
            mean_ground_truth = np.mean(ground_truth_scores)
            stdev_ground_truth = np.std(ground_truth_scores)
            gt_ys.append(mean_ground_truth)
            gt_errors.append(stdev_ground_truth)

            model_scores = data['model_scores']
            mean_model = np.mean(model_scores)
            stdev_model = np.std(model_scores)
            median_ys.append(mean_model)
            median_errors.append(stdev_model)

            abs_diff = np.abs(model_scores - ground_truth_scores)
            mean_abs_diff = np.mean(abs_diff)
            mean_abs_diff_percent = mean_abs_diff / mean_ground_truth
            print("mean_abs_diff_percent", mean_abs_diff_percent)

        ax.errorbar(xs, median_ys, fmt=fmt[0], label=f'Median Agent', markersize=20, linewidth=3)
        ax.errorbar(xs, gt_ys, fmt=fmt[1], label=f'Test Dataset', markersize=10, linewidth=3)
    if legend:
        ax.legend(loc='center left', bbox_to_anchor=(1.1, 0.5))

In [ ]:
fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, sharey=True, figsize=(22,4))
plot(ax1, {"Adult": adult_client_data}, "# of Agents", ylabel=True)
plot(ax2, {"KDD": kdd_client_data}, "# of Agents")
plot(ax3, {"Adult": adult_epsilon_data},"$\epsilon$", logscale=True)
plot(ax4, {"KDD": kdd_epsilon_data}, "$\epsilon$", logscale=True, legend=True)


# plot(ax3, ax4, {"Adult": adult_epsilon_data, "KDD": kdd_epsilon_data}, "$\epsilon$", logscale=True, legend=True)
plt.tight_layout()
plt.savefig("exp1.pdf")
plt